# `無母數檢定 - Python實戰：商務資料結構整理`
### 作者：徐子皓
***

## 基礎資料處理

### 導入原始資料

In [1]:
import pandas as pd
data = pd.read_csv('電商交易資料.csv', encoding='utf-8-sig')
data.head()

,顏色,單價,成本,系列,產品,訂單時間,會員,性別,年紀,廣告代號all,尺寸,訂單編號
0,NaN,643.195,394.800,系列4,產品4-1,2016-01-01T03:19:35,B_GSJ_06674,FEMALE,32.0,廣告_YND_pid,無,201714
1,NaN,391.510,225.365,系列4,產品4-2,2016-01-01T03:19:35,B_GSJ_06674,FEMALE,32.0,廣告_YND_pid,無,201714
2,watermelonred,713.930,416.185,系列4,產品4-3,2016-01-01T03:19:35,B_GSJ_06674,FEMALE,32.0,廣告_YND_pid,S,201714
3,NaN,557.655,337.225,系列4,產品4-4,2016-01-01T03:19:35,B_GSJ_06674,FEMALE,32.0,廣告_YND_pid,無,201714
4,white,628.390,366.835,系列4,產品4-3,2016-01-01T03:19:35,B_GSJ_06674,FEMALE,32.0,廣告_YND_pid,S,201714


### 保留特定欄位與系列

In [2]:
# 只留下需要的欄位
data = data[['單價', '成本', '系列', '廣告代號all']]
# 取系列4作分析
data = data[data['系列']=='系列4']
data.head()

,單價,成本,系列,廣告代號all
0,643.195,394.800,系列4,廣告_YND_pid
1,391.510,225.365,系列4,廣告_YND_pid
2,713.930,416.185,系列4,廣告_YND_pid
3,557.655,337.225,系列4,廣告_YND_pid
4,628.390,366.835,系列4,廣告_YND_pid


## 挑選出廣告系列

### 以B2K系列為例，查看有多少類別的子廣告

In [3]:
for ad in data['廣告代號all'].unique():
    if '_B2K_' in ad:
        print(ad)

廣告_B2K_pid
廣告_B2K_mqrukisleeve
廣告_B2K_mqrukisleeve2
廣告_B2K_MK
廣告_B2K_ciccicqrtu
廣告_B2K_mqrukitrqveltite
廣告_B2K_ciccicqrtu2
廣告_B2K_jqejqe
廣告_B2K_cic2016


### 挑選出廣告系列

In [4]:
ad_series = []
for ad in data['廣告代號all']:
    # 取得系列名稱
    series = ad.split('_')[1]
    # 儲存至串列中
    ad_series.append(series)
ad_series[:10]

['YND', 'YND', 'YND', 'YND', 'YND', 'YND', 'YND', '自然流量', '自然流量', '自然流量']

In [5]:
data['廣告代號'] = ad_series
print('廣告系列代號共有',data['廣告代號'].nunique(),'種')
data.head()

廣告系列代號共有 36 種


,單價,成本,系列,廣告代號all,廣告代號
0,643.195,394.800,系列4,廣告_YND_pid,YND
1,391.510,225.365,系列4,廣告_YND_pid,YND
2,713.930,416.185,系列4,廣告_YND_pid,YND
3,557.655,337.225,系列4,廣告_YND_pid,YND
4,628.390,366.835,系列4,廣告_YND_pid,YND


### 挑選出主力廣告系列

In [6]:
from collections import Counter
count_list = Counter(data['廣告代號']) # 廣告頻率清單
count_frame = pd.DataFrame.from_dict(count_list,orient='index').reset_index() # 轉換成DataFrame
count_frame = count_frame.rename(columns = {'index' : '廣告名稱', 0: '樣本數'}) # 重新命名欄位名稱
count_frame = count_frame.sort_values(by='樣本數', ascending=False) # 按照樣本數，由大至小排序
main_ads = count_frame['廣告名稱'].tolist()[:3] # 取出主力廣告
print('前三主力廣告為：',main_ads)
count_frame.head()

前三主力廣告為： ['自然流量', 'KDP', 'critei']


,廣告名稱,樣本數
1,自然流量,3780
13,KDP,1793
4,critei,1285
7,KDPOD,937
5,qdwit,896


### 篩選出主力廣告資料

In [7]:
data = data[data['廣告代號'].isin(main_ads)]
data.head()

,單價,成本,系列,廣告代號all,廣告代號
7,1044.575,806.050,系列4,廣告_自然流量,自然流量
8,713.930,416.185,系列4,廣告_自然流量,自然流量
9,1044.575,806.050,系列4,廣告_自然流量,自然流量
10,924.490,697.480,系列4,廣告_自然流量,自然流量
11,454.020,319.130,系列4,廣告_自然流量,自然流量


## 取得完成版已處理資料

### 計算利潤，保留特定欄位

In [8]:
data['利潤'] = data['單價'] - data['成本']
data = data[['廣告代號', '利潤']]
data.head()

,廣告代號,利潤
7,自然流量,238.525
8,自然流量,297.745
9,自然流量,238.525
10,自然流量,227.010
11,自然流量,134.890


***

# `Python實戰：如果蒐集樣本太少，又想優化廣告成效？`
### 作者：徐子皓
***

## 取得不同廣告系列的交易利潤資料

In [9]:
ad1_list = round(data[data['廣告代號'] == '自然流量']['利潤'], 2).tolist()
ad2_list = round(data[data['廣告代號'] == 'KDP']['利潤'], 2).tolist()
ad3_list = round(data[data['廣告代號'] == 'critei']['利潤'], 2).tolist()
print('廣告系列「自然流量」的前五筆交易利潤為:', ad1_list[:5])
print('廣告系列「KDP」的前五筆交易利潤為:', ad2_list[:5])
print('廣告系列「critei」的前五筆交易利潤為:', ad3_list[:5])

廣告系列「自然流量」的前五筆交易利潤為: [238.53, 297.74, 238.53, 227.01, 134.89]
廣告系列「KDP」的前五筆交易利潤為: [416.18, 416.18, 329.0, 375.06, 514.88]
廣告系列「critei」的前五筆交易利潤為: [297.74, 261.56, 297.74, 148.05, 227.01]


## 事前檢定

### 常態性檢定

In [10]:
import scipy.stats
print('廣告系列「自然流量」的常態性檢定結果：')
print(scipy.stats.shapiro(ad1_list), '\n')
print('廣告系列「KDP」的常態性檢定結果：')
print(scipy.stats.shapiro(ad2_list), '\n')
print('廣告系列「critei」的常態性檢定結果：')
print(scipy.stats.shapiro(ad3_list))

廣告系列「自然流量」的常態性檢定結果：
ShapiroResult(statistic=0.9822747111320496, pvalue=2.2997640512891535e-21) 

廣告系列「KDP」的常態性檢定結果：
ShapiroResult(statistic=0.9621834754943848, pvalue=3.1784354452832567e-21) 

廣告系列「critei」的常態性檢定結果：
ShapiroResult(statistic=0.9818152189254761, pvalue=1.2428569458322603e-11)


## 無母數檢定

In [11]:
from scipy.stats.mstats import kruskalwallis
kruskalwallis(ad1_list, ad2_list, ad3_list)

KruskalResult(statistic=4.5912367021219875, pvalue=0.1006991066133633)

## 視覺化呈現

### 繪圖

In [12]:
import plotly.offline as py
import plotly.graph_objects as go
from numpy import mean
fig = go.Figure()
fig.add_trace(go.Scatter(
            x= ['critei','KDP','自然流量'],
            y= [mean(ad1_list), mean(ad2_list), mean(ad3_list)],
            mode="lines+markers",
            textfont=dict(
            family="sans serif",
            size=16,
            color="royalblue"),    
            line=dict(color='royalblue', width=2),
            ))

### 佈景主題設定

In [13]:
fig.update_layout(
    title={
        'text': "<b>無母數檢定－廣告效益比較圖</b>",
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',},
    yaxis_title='平均交易利潤',
    xaxis={
        'title': '廣告',
        'tickmode': 'linear'
        },
    width=1800,
    height=960,
    font=dict(
        family="Courier New, monospace",
        size=20,
        color="lightslategrey"
    )
    )

### 將產出另存新檔

In [14]:
# 另存互動式網頁檔案
py.plot(fig, filename='CH4-19產出成果_廣告效益比較圖', auto_open=True)
# 另存圖檔
fig.write_image("CH4-19產出成果_廣告效益分析.png")

C:\Users\user\AppData\Local\Programs\Python\Python37\lib\site-packages\plotly\offline\offline.py:563: UserWarning:

Your filename `CH4-19產出成果_廣告效益比較圖` didn't end with .html. Adding .html to the end of your file.

